In [2]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')

In [3]:
import matplotlib
import matplotlib.pyplot as plt
plt.rcParams['font.family'] = 'NanumGothic'

matplotlib.rc("axes", unicode_minus = False)

In [4]:
df1 = pd.read_csv("01 Oxidation.csv")
df2 = pd.read_csv("02 Photo_softbake.csv")
df3 = pd.read_csv("03 Photo_lithograpy.csv")
df4 = pd.read_csv("04 Etching.csv")
df5 = pd.read_csv("05 Ion_Implantation.csv")
df6 = pd.read_csv("06 Inspect.csv")

In [5]:
df = pd.merge(df1, df2, on=['No_Die','Lot_Num', 'Wafer_Num','Datetime'])
df = pd.merge(df, df3, on=['No_Die','Lot_Num', 'Wafer_Num','Datetime'])
df = pd.merge(df, df4, on=['No_Die','Lot_Num', 'Wafer_Num','Datetime'])
df = pd.merge(df, df5, on=['No_Die','Lot_Num', 'Wafer_Num','Datetime'])
df = pd.merge(df, df6, on=['No_Die','Lot_Num', 'Wafer_Num','Datetime'])

In [6]:
df.dropna(subset="Thin F2", inplace=True) # 다른 변수들과의 연관성을 찾지못함, 한 행에 여러 열들 결측값 가짐
# pd.set_option('display.max_row', 200)
# pd.set_option('display.max_columns', 200)

In [7]:
df['Ox_Chamber'] = df['Ox_Chamber'].astype('str')
df['photo_soft_Chamber'] = df['photo_soft_Chamber'].astype('str')
df['lithography_Chamber'] = df['lithography_Chamber'].astype('str')
df['Etching_Chamber'] = df['Etching_Chamber'].astype('str')
df['Chamber_Num'] = df['Chamber_Num'].astype('str')
df['path'] = df['Ox_Chamber']+df['photo_soft_Chamber']+df['lithography_Chamber']+df['Etching_Chamber']+df['Chamber_Num']

In [8]:
df.drop(df[df['Oxid_time']<0].index, inplace=True) # 산화시간이 음수
df.drop(df[df['Target']==0].index, inplace=True) # target 값이 0
df.drop(columns = ["Wafer_map","Error_message"],inplace=True) # 웨이퍼맵, 에러메시지

In [9]:
# 시간 데이터 datetime 유형으로 변환
df['Datetime'] = pd.to_datetime(df['Datetime'], format='%d-%m-%Y')

# 195이상 = 불량(1), 195미만 = 양품(0)
df.loc[df['Target'] >= 195, '불량_195이상'] =1
df.loc[df['Target'] < 195, '불량_195이상'] =0

In [10]:
df.loc[df['Thin F4']<0, 'Thin F4']=df['Thin F4'].median()
df.loc[df['Flux90s']<0, 'Flux90s']=df['Flux90s'].median()
df.loc[df['Flux160s']<0, 'Flux160s']=df['Flux160s'].median()
df.loc[df['Flux160s']<5, 'Flux160s']=df['Flux160s'].median()

In [11]:
df['Thin F1'].fillna(df['Thin F1'].median(), inplace=True)
df['Thin F3'].fillna(df['Thin F3'].median(), inplace=True)
df['Flux60s'].fillna(df['Flux90s'].median(), inplace=True)
df['Flux90s'].fillna(df['Flux90s'].median(), inplace=True)
df['Flux480s'].fillna(df['Flux90s'].median(), inplace=True)
df['Flux840s'].fillna(df['Flux90s'].median(), inplace=True)

### 불필요한 열 제거

In [12]:
# 불필요한 열 제거 
# df.drop(columns=["No_Die", "Lot_Num", "Wafer_Num"], inplace=True)
df.drop(columns=['Vapor','process','Wavelength'])
df.loc[df['Flux840s'] == 8.137500e+16, 'Flux840s'] = df['Flux840s'].mode()[0]
df.loc[df['Flux480s'] == 8.137500e+16, 'Flux480s'] = df['Flux480s'].mode()[0]

In [13]:
df_new = df[['Temp_OXid','ppm','Pressure','type','Oxid_time',
        'N2_HMDS', 'pressure_HMDS', 'temp_HMDS', 'temp_HMDS_bake',
       'time_HMDS_bake', 'spin1', 'spin2', 'spin3', 'photoresist_bake',
       'temp_softbake', 'time_softbake', 'UV_type', 'Energy_Exposure',       
        'Temp_Etching', 'Source_Power',
       'input_Energy', 'Temp_implantation', 'Furance_Temp', 'RTA_Temp',
       '불량_195이상', 'path']]

In [14]:
import xgboost as xgb
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score

# 데이터 구성:Series, DataFrame
import pandas as pd
# 데이터 시각화
import matplotlib.pyplot as plt
import matplotlib

# 데이터 분할:train, test
from sklearn.model_selection import train_test_split
# 분류 Gradient Boosting
from sklearn.ensemble import GradientBoostingClassifier
# 최적 모델, 파라미터 탐색
from sklearn.model_selection import GridSearchCV

# 분류모델 평가 함수
from sklearn.metrics import accuracy_score, roc_auc_score, precision_score, recall_score, f1_score
from sklearn.metrics import confusion_matrix, classification_report

In [15]:
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import MinMaxScaler
# LabelEncoder 객체 생성
label_encoder = LabelEncoder()

df_raw_x = df_new.drop("불량_195이상", axis = 1)
df_raw_y = df_new["불량_195이상"] 

# MinMaxScaler 객체 생성
numeric_columns = df_raw_x.select_dtypes(include=['number']).columns
scaler = MinMaxScaler()

df_raw_x[numeric_columns] = scaler.fit_transform(df_raw_x[numeric_columns])

df_raw_x = pd.get_dummies(df_raw_x, columns=df_raw_x.select_dtypes(include=['object']).columns)
df_raw_x = df_raw_x.astype({column: int for column in df_raw_x.select_dtypes(include=['bool']).columns})

df_train_x, df_test_x, df_train_y, df_test_y = train_test_split(
    df_raw_x, df_raw_y, test_size = 0.3, random_state = 1234, stratify=df_raw_y) 

In [16]:
xgb_model = xgb.XGBClassifier(random_state=1234)

xgb_model.fit(df_train_x, df_train_y)
# Train 데이터 설명력
print("Score on training set: {:.3f}".format(xgb_model.score(df_train_x, df_train_y)))
# Test 데이터 설명력
print("Score on test set: {:.3f}".format(xgb_model.score(df_test_x, df_test_y)))

Score on training set: 1.000
Score on test set: 0.963


In [17]:
from sklearn.model_selection import RandomizedSearchCV
from xgboost import XGBClassifier
from scipy.stats import uniform, randint

# XGBoost 모델 정의
xgb_model = XGBClassifier()

# 탐색할 하이퍼파라미터의 분포 설정
param_dist = {
    'reg_lambda': uniform(1e-8, 1e-7 - 1e-8),
    'reg_alpha': uniform(1e-8, 1e-7 - 1e-8),
    'learning_rate': [0.001, 0.01, 0.05, 0.1],
    'n_estimators': randint(100, 301),
    'max_depth': randint(3, 10)
}

# 랜덤 서치 객체 생성
random_search = RandomizedSearchCV(estimator=xgb_model, param_distributions=param_dist, scoring='accuracy',
                                   cv=10, n_iter=100)

# 랜덤 서치 수행
random_search.fit(df_train_x, df_train_y)

# 최적의 하이퍼파라미터 출력
print("Best parameters found: ", random_search.best_params_)
print("best estimator model: \n{}".format(random_search.best_estimator_))
# 최적의 모델
best_xgb_model = random_search.best_estimator_

# 예측 수행
train_pred = best_xgb_model.predict(df_train_x)
test_pred = best_xgb_model.predict(df_test_x)

Best parameters found:  {'learning_rate': 0.05, 'max_depth': 6, 'n_estimators': 242, 'reg_alpha': 5.551695018865267e-08, 'reg_lambda': 7.513887184563936e-08}


In [ ]:
print("best estimator model: \n{}".format(random_search.best_estimator_))
print("\nbest parameter: \n{}".format(random_search.best_params_))
print("\nbest score: \n{}".format(random_search.best_score_.round(3)))

In [18]:
xgb_model = xgb.XGBClassifier(learning_rate=0.5,max_depth=6 ,n_estimators= 242,reg_alpha= 5.551695018865267e-08
                              ,reg_lambda= 7.513887184563936e-08, random_state=1234)
xgb_model.fit(df_train_x, df_train_y)
y_pred = xgb_model.predict(df_test_x)
# Train 데이터 설명력
print("Score on training set: {:.3f}".format(xgb_model.score(df_train_x, df_train_y)))
# Test 데이터 설명력
print("Score on test set: {:.3f}".format(xgb_model.score(df_test_x, df_test_y)))

print("Accuracy on training set: {:.3f}".format(xgb_model.score(df_train_x, df_train_y)))
print("Accuracy on test set: {:.3f}\n".format(xgb_model.score(df_test_x, df_test_y)))
print("Confusion matrix: \n{}".format(confusion_matrix(df_test_y, y_pred)))

print(classification_report(df_test_y, y_pred, digits=3))

Score on training set: 1.000
Score on test set: 0.967
Accuracy on training set: 1.000
Accuracy on test set: 0.967

Confusion matrix: 
[[470   2]
 [ 15  21]]
              precision    recall  f1-score   support

         0.0      0.969     0.996     0.982       472
         1.0      0.913     0.583     0.712        36

    accuracy                          0.967       508
   macro avg      0.941     0.790     0.847       508
weighted avg      0.965     0.967     0.963       508



In [ ]:
v_feature_names = df_train_x.columns

df_importance = pd.DataFrame()
df_importance ["Feature"] = v_feature_names
df_importance ["Importance"] = xgb_model.feature_importances_
df_importance .sort_values("Importance", ascending=False, inplace = True)
df_importance .round(3)

In [ ]:
df_importance = pd.DataFrame()
df_importance["Feature"] = v_feature_names
df_importance["Importance"] = xgb_model.feature_importances_

# df_feature_importance의 테이블을 중요도별로 정렬
df_importance.sort_values("Importance", ascending=False, inplace = True)
df_importance.sort_values("Importance", ascending=True, inplace = True)
coordinates = range(len(df_importance))
top_10_importance = df_importance.nlargest(10, "Importance")

# 시각화
coordinates = range(len(top_10_importance))
plt.barh(y=coordinates[::-1], width=top_10_importance["Importance"])
plt.yticks(coordinates[::-1], top_10_importance["Feature"], fontsize=10)
plt.xlabel("변수 중요도")
plt.ylabel("변수")
plt.title("상위 10개 변수 중요도")
plt.show()

### SMOTE

In [19]:
from imblearn.over_sampling import SMOTE
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report

In [20]:
smote = SMOTE(random_state=42)
X_resampled, y_resampled = smote.fit_resample(df_train_x, df_train_y)
xgb_clf = xgb.XGBClassifier(random_state=42)
xgb_clf.fit(X_resampled, y_resampled)

y_pred = xgb_clf.predict(df_test_x)
print(classification_report(df_test_y, y_pred))

              precision    recall  f1-score   support

         0.0       0.97      0.99      0.98       472
         1.0       0.77      0.67      0.72        36

    accuracy                           0.96       508
   macro avg       0.87      0.83      0.85       508
weighted avg       0.96      0.96      0.96       508



In [23]:
from sklearn.model_selection import RandomizedSearchCV
from xgboost import XGBClassifier
from scipy.stats import uniform, randint

# XGBoost 모델 정의
xgb_model = XGBClassifier()

# 탐색할 하이퍼파라미터의 분포 설정
param_dist = {
    'reg_lambda': uniform(1e-8, 0.1),
    'reg_alpha': uniform(1e-8, 0.1),
    'learning_rate': uniform(1e-4, 0.1),
    'n_estimators': randint(100, 301),
    'max_depth': randint(3, 10),
    'gamma': uniform(0, 0.5)
}

# 랜덤 서치 객체 생성
random_search = RandomizedSearchCV(estimator=xgb_model, param_distributions=param_dist, scoring='accuracy',
                                   cv=10, n_iter=200)

# 랜덤 서치 수행
random_search.fit(X_resampled, y_resampled)

best_xgb_model = random_search.best_estimator_
print("best estimator model: \n{}".format(random_search.best_estimator_))
print("\nbest parameter: \n{}".format(random_search.best_params_))
print("\nbest score: \n{}".format(random_search.best_score_.round(3)))

best estimator model: 
XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, device=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              gamma=0.12620180126392483, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=0.07433674592574852,
              max_bin=None, max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=6, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              multi_strategy=None, n_estimators=181, n_jobs=None,
              num_parallel_tree=None, random_state=None, ...)

best parameter: 
{'gamma': 0.12620180126392483, 'learning_rate': 0.07433674592574852, 'max_depth': 6, 'n_estimators': 181, 'reg_alpha': 0.08883657957087174, 'reg_lambda': 0.05472367005012898}


In [24]:
xgb_model = xgb.XGBClassifier(learning_rate=0.07433674592574852,max_depth=6 ,n_estimators= 181,
                              reg_alpha= 0.08883657957087174,reg_lambda= 0.05472367005012898, 
                              gamma = 0.12620180126392483,
                              random_state=1234)
xgb_model.fit(X_resampled, y_resampled)

y_pred = xgb_model.predict(df_test_x)

print("Confusion matrix: \n{}".format(confusion_matrix(df_test_y, y_pred)))
print(classification_report(df_test_y, y_pred))

Confusion matrix: 
[[467   5]
 [ 13  23]]
              precision    recall  f1-score   support

         0.0       0.97      0.99      0.98       472
         1.0       0.82      0.64      0.72        36

    accuracy                           0.96       508
   macro avg       0.90      0.81      0.85       508
weighted avg       0.96      0.96      0.96       508



In [25]:
df_new.isnull().sum()

Temp_OXid            0
ppm                  0
Pressure             0
type                 0
Oxid_time            0
N2_HMDS              0
pressure_HMDS        0
temp_HMDS            0
temp_HMDS_bake       0
time_HMDS_bake       0
spin1                0
spin2                0
spin3                0
photoresist_bake     0
temp_softbake        0
time_softbake        0
UV_type              0
Energy_Exposure      0
Temp_Etching         0
Source_Power         0
input_Energy         0
Temp_implantation    0
Furance_Temp         0
RTA_Temp             0
불량_195이상             0
path                 0
dtype: int64